# Coroutine

In [1]:
def A():
    print('1')
    print('2')
    print('3')
def B():
    print('x')
    print('y')
    print('z')

generator 

In [12]:
def consumer():
    r=''
    while True:
        n=yield r 
        if not n:
            return
        print('[CONSUMER] Consuming %s...' %n)
        r='200 OK'
def produce(c):
    c.send(None)
    n=0
    while n<5:
        n=n+1 
        print('[PRODUCER] Producing %s...' %n)
        r=c.send(n)
        print('[PRODUCER] Consumer return: %s' % r)
    c.close()

In [13]:
c=consumer()
produce(c)

[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 5...
[CONSUMER] Consuming 5...
[PRODUCER] Consumer return: 200 OK


# asyncio

In [1]:
import asyncio

@asyncio.coroutine
def hello():
    print("hello world!")
    r= yield from asyncio.sleep(1)
    print("hello again!")

In [2]:
loop = asyncio.get_event_loop()

In [3]:
loop.run_until_complete(hello())
loop.close()

hello world!
hello again!


In [1]:
import threading
import asyncio

@asyncio.coroutine
def hello():
    print('Hello world! (%s)' % threading.currentThread())
    yield from asyncio.sleep(1)
    print('Hello again! (%s)' % threading.currentThread())

loop2 = asyncio.get_event_loop()
tasks=[hello(),hello()]
loop2.run_until_complete(asyncio.wait(tasks))
loop2.close()

Hello world! (<_MainThread(MainThread, started 13960)>)
Hello world! (<_MainThread(MainThread, started 13960)>)
Hello again! (<_MainThread(MainThread, started 13960)>)
Hello again! (<_MainThread(MainThread, started 13960)>)


In [1]:
import asyncio
@asyncio.coroutine
def wget(host):
    print('wget %s...' % host)
    connect=asyncio.open_connection(host,80)
    reader,writer=yield from connect
    header='GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host,line.decode('utf-8').rstrip()))
    writer.close()

In [2]:
loop=asyncio.get_event_loop()
tasks=[wget(host) for host in ['www.sina.com.cn','www.sohu.com','www.163.com']]

In [3]:
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

wget www.sina.com.cn...
wget www.163.com...
wget www.sohu.com...
www.sina.com.cn header > HTTP/1.1 302 Moved Temporarily
www.sina.com.cn header > Server: nginx
www.sina.com.cn header > Date: Sat, 23 Feb 2019 05:57:52 GMT
www.sina.com.cn header > Content-Type: text/html
www.sina.com.cn header > Content-Length: 154
www.sina.com.cn header > Connection: close
www.sina.com.cn header > Location: https://www.sina.com.cn/
www.sina.com.cn header > X-Via-CDN: f=edge,s=cern.beijing.ha2ts4.211.nb.sinaedge.com,c=2001:da8:8001:833:54db:62d6:9115:4ed2;
www.sina.com.cn header > X-Via-Edge: 155090147294700541591410011021eabd476
www.sohu.com header > HTTP/1.1 200 OK
www.sohu.com header > Content-Type: text/html;charset=UTF-8
www.sohu.com header > Connection: close
www.sohu.com header > Server: nginx
www.sohu.com header > Date: Sat, 23 Feb 2019 05:57:29 GMT
www.sohu.com header > Cache-Control: max-age=60
www.sohu.com header > X-From-Sohu: X-SRC-Cached
www.sohu.com header > Content-Encoding: gzip
www.sohu

# async/await

@asyncio.coroutine ->> async;

yield from ->> await

In [1]:
import asyncio
async def wget(host):
    print('wget %s...' % host)
    connect=asyncio.open_connection(host,80)
    reader,writer=await connect
    header='GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    await writer.drain()
    while True:
        line = await reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host,line.decode('utf-8').rstrip()))
    writer.close()

loop=asyncio.get_event_loop()
tasks=[wget(host) for host in ['www.sina.com.cn','www.sohu.com','www.163.com']]

loop.run_until_complete(asyncio.wait(tasks))
loop.close()

wget www.sina.com.cn...
wget www.sohu.com...
wget www.163.com...
www.sina.com.cn header > HTTP/1.1 302 Moved Temporarily
www.sina.com.cn header > Server: nginx
www.sina.com.cn header > Date: Sat, 23 Feb 2019 06:04:08 GMT
www.sina.com.cn header > Content-Type: text/html
www.sina.com.cn header > Content-Length: 154
www.sina.com.cn header > Connection: close
www.sina.com.cn header > Location: https://www.sina.com.cn/
www.sina.com.cn header > X-Via-CDN: f=edge,s=cern.beijing.ha2ts4.206.nb.sinaedge.com,c=2001:da8:8001:833:54db:62d6:9115:4ed2;
www.sina.com.cn header > X-Via-Edge: 1550901848898005415914100060277f539b2
www.sohu.com header > HTTP/1.1 200 OK
www.sohu.com header > Content-Type: text/html;charset=UTF-8
www.sohu.com header > Connection: close
www.sohu.com header > Server: nginx
www.sohu.com header > Date: Sat, 23 Feb 2019 06:03:35 GMT
www.sohu.com header > Cache-Control: max-age=60
www.sohu.com header > X-From-Sohu: X-SRC-Cached
www.sohu.com header > Content-Encoding: gzip
www.sohu

In [1]:
import asyncio
import time

async def say_after(delay,what):
    await asyncio.sleep(delay)
    print(what)
    
async def main():
    print(f"started at {time.strftime('%X')}")
    await say_after(1,'hello')
    await say_after(2,'world')
    print(f"finished at {time.strftime('%X')}")

In [2]:
if __name__=='__main__':
    loop=asyncio.get_event_loop()
    loop.run_until_complete(main())
    loop.close()

started at 08:55:28
hello
world
finished at 08:55:31


# Awaitables Object

## Coroutines

In [5]:
async def nested():
    return 42 
async def main():
    nested()
    print(await nested())
loop=asyncio.new_event_loop()
asyncio.set_event_loop(loop)
loop.run_until_complete(main())
loop.close()

42


d:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: coroutine 'nested' was never awaited
  after removing the cwd from sys.path.


*nested()* will not run because it's not *__awaited__*, but *_await_ nested()* will print __42__.

## Tasks

In [22]:
async def nested():
    await asyncio.sleep(2)
    print("waiting 2s")
async def main():
    task=asyncio.ensure_future(nested())
    print(task)
    await task
    print(task.done())
if __name__=='__main__':
    loop=asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        loop.run_until_complete(main())
    except KeyboardInterrupt as e:
        for task in asyncio.Task.all_tasks():
            print(task)
            task.cancel()
            print(task)
        loop.run_forever()
    finally:
        loop.close()

<Task pending coro=<nested() running at <ipython-input-22-73d8e0000cbf>:1>>
waiting 2s
True


## Future

# Gather tasks

## asyncio.gather

In [6]:
async def factorial(name,number):
    f=1 
    for i in range(2,number+1):
        print(f"Task {name}: Compute factorial({i})...")
        if number == 2:
            1/0
        await asyncio.sleep(1)
        f *= i 
    print(f"Task {name}: factorial({number}) = {f}")

In [7]:
async def main():
    res = await asyncio.gather(
    *[factorial("A",2),
     factorial("B",3),
     factorial("C",4)],
    return_exceptions=True)
    for item in res:
        print(item)

In [8]:
loop=asyncio.new_event_loop()
asyncio.set_event_loop(loop)
try:
    loop.run_until_complete(main())
except KeyboardInterrupt as e: 
    for task in asyncio.Task.all_tasks():
        print(task)
        task.cancel()
        print(task)
    loop.run_forever()
finally:
    loop.close()

Task B: Compute factorial(2)...
Task C: Compute factorial(2)...
Task A: Compute factorial(2)...
Task B: Compute factorial(3)...
Task C: Compute factorial(3)...
Task B: factorial(3) = 6
Task C: Compute factorial(4)...
Task C: factorial(4) = 24
division by zero
None
None


## asyncio.wait

# Timeouts

asyncio.wait_for()

asyncio.wait_for()

In [10]:
async def eternity():
    await asyncio.sleep(3600)
    print('yay!')
async def main():
    try:
        await asyncio.wait_for(eternity(),timeout=1.0)
    except asyncio.TimeoutError:
        print('timeout!')
loop=asyncio.new_event_loop()
asyncio.set_event_loop(loop)
loop.run_until_complete(main())

timeout!


asyncio.as_complete()

In [ ]:
tasks=[asyncio.ensure_future(coroutine1),
      asyncio.ensure_future(coroutine2),
      asyncio.ensure_future(coroutine3)]
for task in asyncio.as_completed(tasks):
    pass

# Thread Scheduling

In [11]:
import threading

In [ ]:
thre

In [16]:
def start_loop(loop):
    asyncio.set_event_loop(loop)
    loop.run_forever()
async def do_some_work(x):
    print('Waiting {}'.format(x))
    await asyncio.sleep(x)
    print('Done after {}s'.format(x))
def more_work(x):
    print('More work {}'.format(x))
    time.sleep(x)
    print('Finished more work {}'.format(x))
start=time.time()
new_loop=asyncio.new_event_loop()
t=threading.Thread(target=start_loop,args=(new_loop,))
t.start()
print('TIME: {}'.format(time.time()-start))
asyncio.run_coroutine_threadsafe(do_some_work(6),new_loop)
asyncio.run_coroutine_threadsafe(do_some_work(4),new_loop)

TIME: 0.0


<Future at 0x1eeff6df518 state=pending>

Waiting 6
Waiting 4
Done after 4s
Done after 6s


run_in_executor

In [22]:
async def main():
    print(f'{time.ctime()} Hello')
    await asyncio.sleep(1.0)
    print(f'{time.ctime()} Goodbye')
    loop.stop()

In [23]:
def blocking():
    time.sleep(0.5)
    print(f'{time.ctime()} Hello from a thread!')

In [18]:
t

<Thread(Thread-6, started 17196)>

In [25]:
loop=asyncio.new_event_loop()
asyncio.set_event_loop(loop)
loop.create_task(main())
loop.run_in_executor(None,blocking)
loop.run_forever()
pending=asyncio.Task.all_tasks(loop=loop)
group=asyncio.gather(*pending)
loop.run_until_complete(group)
loop.close()

Sun Feb 24 10:53:34 2019 Hello
Sun Feb 24 10:53:34 2019 Hello from a thread!
Sun Feb 24 10:53:35 2019 Goodbye


*loop.run_in_executor()* use *executor* solve blocking

# Binding callback

In [31]:
def callback(future):
    print('Callback: ', future.result())
now=lambda: time.time()

In [34]:
start=now()
coroutine=do_some_work(2)
loop=asyncio.new_event_loop()
asyncio.set_event_loop(loop)
get_future=asyncio.ensure_future(coroutine)
get_future.add_done_callback(callback)
loop.run_until_complete(get_future)
print('TIME: ', now()-start)

Waiting 2
Done after 2s
Callback:  None
TIME:  2.000216484069824


When task completed, execute callback.

In [35]:
import functools
def callback(t,future):
    print('Callback:',t,future.result())
get_future.add_done_callback(functools.partial(callback,2))

In [39]:
loop.run_until_complete(get_future)

if callback with one more parameters, use **functools.partial**.